# Intermediate Linear Regression Practice

## Use a Linear Regression model to get the lowest RMSE possible on the following dataset:

[Dataset Folder](https://github.com/ryanleeallred/datasets/tree/master/Ames%20Housing%20Data)

[Raw CSV](https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv)

## You model must include (at least):
- A log-transformed y variable
- Two polynomial features
- One interaction feature
- 10 other engineered features

What is the lowest Root-Mean-Squared Error that you are able to obtain? Share your best RMSEs in Slack!

Notes:

There may be some data cleaning that you need to do on some features of this dataset. Linear Regression will only accept numeric values and will not accept

Note* There may not be a clear candidate for an interaction term in this dataset. Include one anyway, sometimes it's a good practice for predictive modeling feature engineering in general. 

In [1]:
##### Your Code Here #####
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from math import sqrt

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',500)

In [2]:
url = 'https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv'

data = pd.read_csv(url)

print(data.shape)
data.head()


(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
data.isna().sum()

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinSF1          0
BsmtFinType2       38
BsmtFinSF2          0
BsmtUnfSF           0
TotalBsmtSF         0
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
1stFlrSF            0
2ndFlrSF            0
LowQualFin

In [4]:
df = data.drop(columns = ['LotFrontage', 'Alley', 
                          'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
                          'BsmtFinType2', 'FireplaceQu', 'GarageType',
                          'GarageFinish', 'GarageYrBlt','GarageQual','GarageCond', 'PoolQC',
                          'Fence','MiscFeature'])
df.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageCars,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,706,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,2,548,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,978,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,2,460,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,486,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,2,608,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,216,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,3,642,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,655,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,3,836,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


In [5]:
df.fillna(method='ffill')

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageCars,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,706,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,2,548,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,978,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,2,460,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,486,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,2,608,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,216,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,3,642,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,655,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,3,836,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000
5,6,50,RL,14115,Pave,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,Wood,732,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,2,480,Y,40,30,0,320,0,0,700,10,2009,WD,Normal,143000
6,7,20,RL,10084,Pave,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,1369,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,2,636,Y,255,57,0,0,0,0,0,8,2007,WD,Normal,307000
7,8,60,RL,10382,Pave,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,859,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,2,484,Y,235,204,228,0,0,0,350,11,2009,WD,Normal,200000
8,9,50,RM,6120,Pave,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.0,TA,TA,BrkTil,0,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,2,468,Y,90,0,205,0,0,0,0,4,2008,WD,Abnorml,129900
9,10,190,RL,7420,Pave,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,851,0,140,991,GasA,Ex,Y,SBrkr,1077,0,0,1077,1,0,1,0,2,2,TA,5,Typ,2,1,205,Y,0,4,0,0,0,0,0,1,2008,WD,Normal,118000


In [6]:
df_nums = df.select_dtypes(include=['float','int']).dropna(axis=1)

y = df_nums['SalePrice']
X = df_nums.drop(columns=['Id','SalePrice'])

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=42)

model = LinearRegression()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test,y_pred)
rmse = sqrt(mse)
r2 = r2_score(y_test,y_pred)

beta_0 = model.intercept_
beta_i = model.coef_

print('mean squared error: ', mse)
print('root mean squared error: ', rmse)
print('R squared: ', r2)
print('---------------------')
print('Intercept: ', beta_0)
print('Coefficients: ', beta_i)


mean squared error:  1392205606.2031894
root mean squared error:  37312.27152296131
R squared:  0.8184946026286748
---------------------
Intercept:  -48411.54387291902
Coefficients:  [-1.75548690e+02  4.09174234e-01  1.84955503e+04  3.83573710e+03
  3.37312908e+02  1.80209917e+02  1.00461555e+01 -4.61607099e+00
  3.82786747e-01  5.81287122e+00  1.31240039e+01  1.42134340e+01
  4.50422890e+00  3.18416668e+01  1.10492803e+04 -3.26518493e+02
  3.42606130e+03 -1.66531509e+03 -9.08810085e+03 -1.00971990e+04
  5.10100134e+03  4.37021557e+03  1.16074917e+04  5.58607730e-01
  2.55639049e+01 -6.62562003e+00  7.06908781e+00  3.85262374e+01
  6.55989735e+01 -3.73443286e+01 -7.15692563e-01 -2.09433750e+02
 -5.14524004e+02]


In [7]:
df['SaleCondition'].unique()

array(['Normal', 'Abnorml', 'Partial', 'AdjLand', 'Alloca', 'Family'],
      dtype=object)

In [8]:
mask = {
    'Abnorml': 1, 
    'Partial': 2,
    'Normal': 3, 
    'AdjLand': 5,
    'Alloca': 4,
    'Family': 6,
}

df['SaleCondition'] = df['SaleCondition'].replace(mask,inplace=True)

In [9]:
print((df['Heating'].unique()))
print((df['HeatingQC'].unique()))
print((df['Foundation'].unique()))

foundations = {
    'PConc': 1,
    'CBlock': 2, 
    'BrkTil': 3,
    'Wood': 4,
    'Slab': 5, 
    'Stone': 6,
}
heating = {
    'Ex': 1,
    'Gd': 2,
    'TA': 3, 
    'Fa': 4,
    'Po': 5,
    
}
type_heating = {
    'GasA': 1,
    'GasW': 2, 
    'Grav': 3, 
    'Wall': 4,
    'OthW': 5,
    'Floor': 6
}
# df['N_Foundation'] = df['Foundation'].replace(foundations)
# df['N_HeatingQC'] = df['HeatingQC'].replace(heating)
# df['N_Heating'] = df['Heating'].replace(type_heating)

['GasA' 'GasW' 'Grav' 'Wall' 'OthW' 'Floor']
['Ex' 'Gd' 'TA' 'Fa' 'Po']
['PConc' 'CBlock' 'BrkTil' 'Wood' 'Slab' 'Stone']


In [10]:
# all_the_ones = ['Ones_Foundation','Ones_HeatingQC','Ones_Heating']

In [11]:
df['ln_price'] = np.log(df['SalePrice'])
df['N_Baths'] = df['BsmtFullBath'] + 0.5 * df['BsmtHalfBath'] + df['FullBath'] + 0.5 * df['HalfBath']
df['AboveGround'] = df['1stFlrSF'] + df['2ndFlrSF']
# df['Sale_Weight_Condition'] = df['ln_price'] * df['SaleCondition']
df['Quality_Overall'] = np.divide((df['OverallQual'] * df['OverallCond']), ((df['YrSold'] - df['YearRemodAdd'])+df['YrSold']-df['YearBuilt']) + df['YearBuilt'])
# df['Foundation_Weight'] = df['ln_price'] * df['Ones_Foundation']
# df['HeatingQC_Weight'] = df['Ones_HeatingQC'] * df['ln_price']
# df['HeatingType_Weight'] = df['Ones_Heating'] * df['ln_price']


In [12]:
## log
df_new = df.select_dtypes(include=['float','int']).dropna(axis=1)

y = df_new['ln_price']
X = df_new.drop(columns=['SalePrice','ln_price','Id',
                         'BsmtFullBath','BsmtHalfBath','FullBath',
                         'HalfBath','1stFlrSF','2ndFlrSF',
                         'OverallQual','OverallCond'])

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=42)

model = LinearRegression()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test,y_pred)
rmse = sqrt(mse)
r2 = r2_score(y_test,y_pred)

beta_0 = model.intercept_
beta_i = model.coef_

print('mean squared error: ', mse)
print('root mean squared error: ', rmse)
print('R squared: ', r2)
print('---------------------')
print('Intercept: ', beta_0)
print('Coefficients: ', beta_i)

mean squared error:  0.02272951955471559
root mean squared error:  0.1507631239883135
R squared:  0.878200037338549
---------------------
Intercept:  13.228056522477532
Coefficients:  [-5.98173630e-04  1.57014662e-06  3.42634477e-03  9.85859491e-04
  3.74003619e-05  1.35222297e-05  1.48721066e-05  6.57892673e-05
  9.67544849e-05  1.32847007e-04 -9.94861335e-03 -6.91207102e-02
  1.93272182e-02  5.37997784e-02  8.01156310e-02  2.43980400e-05
  1.18696568e-04 -6.04442249e-05  2.46231615e-04  2.57570418e-04
  3.29148107e-04 -4.25412127e-04 -5.48663030e-06  8.80637797e-04
 -5.46356291e-03  5.56797943e-02  3.60924867e-05  2.17230463e+01]


In [14]:
pf5 = PolynomialFeatures(degree=5, interaction_only=False,include_bias=False)
pf6 = PolynomialFeatures(degree=6, interaction_only=False,include_bias=False)
pf3 = PolynomialFeatures(degree=3)
pf2 = PolynomialFeatures(degree=2)

#okay this is from some tutorial ima try it
# test = df[['N_HeatingQC','N_Heating']]
# test2 = pf2.fit_transform(test)
#didn't work lmao

In [15]:
df.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageCars,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,ln_price,N_Baths,AboveGround,Quality_Overall
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,706,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,2,548,Y,0,61,0,0,0,0,0,2,2008,WD,None,208500,12.247694,3.5,1710,0.017387
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,978,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,2,460,Y,298,0,0,0,0,0,0,5,2007,WD,None,181500,12.109011,2.5,1262,0.023553
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,486,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,2,608,Y,0,42,0,0,0,0,0,9,2008,WD,None,223500,12.317167,3.5,1786,0.017378
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,216,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,3,642,Y,0,35,272,0,0,0,0,2,2006,WD,None,140000,11.849398,2.0,1717,0.017140
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,655,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,3,836,Y,192,84,0,0,0,0,0,12,2008,WD,None,250000,12.429216,3.5,2198,0.019841


In [16]:
print(df['Condition1'].unique())
print(df['Condition2'].unique())
print(df['Neighborhood'].unique())
print(df['LandSlope'].unique())
print(df['BldgType'].unique())

bldg = {
    '1Fam':1,
    '2fmCon':2,
    'Duplex':3,
    'TwnhsE':4,
    'Twnhs':5
}
land = {
    'Gtl':1,
    'Mod':2,
    'Sev':3
}
# df['N_LandSl'] = df['LandSlope'].replace(land)
# df['N_Bldg'] = df['BldgType'].replace(bldg)

['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
['Gtl' 'Mod' 'Sev']
['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']


In [17]:
le = LabelEncoder()
ohe_0 = OneHotEncoder(categorical_features=[0])

n_cols = ['HeatingQC','Heating', 'LandSlope', 'BldgType', 'Foundation']

lmao = df[n_cols].values
lmao

array([['Ex', 'GasA', 'Gtl', '1Fam', 'PConc'],
       ['Ex', 'GasA', 'Gtl', '1Fam', 'CBlock'],
       ['Ex', 'GasA', 'Gtl', '1Fam', 'PConc'],
       ...,
       ['Ex', 'GasA', 'Gtl', '1Fam', 'Stone'],
       ['Gd', 'GasA', 'Gtl', '1Fam', 'CBlock'],
       ['Gd', 'GasA', 'Gtl', '1Fam', 'CBlock']], dtype=object)

In [38]:
for i in range(5):
    lmao[:,i] = le.fit_transform(lmao[:,i])

type(lmao)

df_le = pd.DataFrame(data=lmao, columns={'LE_HeatingQC','LE_Heating','LE_LandSlope','LE_Bldg','LE_Foundation'})

final_df = pd.DataFrame.join(df,df_le,how='right')
final_df.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageCars,GarageArea,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,ln_price,N_Baths,AboveGround,Quality_Overall,N_HouseStyle,LE_LandSlope,LE_Bldg,LE_HeatingQC,LE_Foundation,LE_Heating
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,706,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,2,548,Y,0,61,0,0,0,0,0,2,2008,WD,None,208500,12.247694,3.5,1710,0.017387,2,0,1,0,0,2
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,978,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,2,460,Y,298,0,0,0,0,0,0,5,2007,WD,None,181500,12.109011,2.5,1262,0.023553,1,0,1,0,0,1
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,486,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,2,608,Y,0,42,0,0,0,0,0,9,2008,WD,None,223500,12.317167,3.5,1786,0.017378,2,0,1,0,0,2
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,216,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,3,642,Y,0,35,272,0,0,0,0,2,2006,WD,None,140000,11.849398,2.0,1717,0.017140,2,2,1,0,0,0
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,655,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,3,836,Y,192,84,0,0,0,0,0,12,2008,WD,None,250000,12.429216,3.5,2198,0.019841,2,0,1,0,0,2


In [40]:
df['HouseStyle'].unique()

style = {
    '2Story': 2, 
    '1Story': 1,
    '1.5Fin': 2,  
    '1.5Unf': 1, 
    'SFoyer': 2, 
    'SLvl': 1, 
    '2.5Unf': 2,
    '2.5Fin': 3
}
df['N_HouseStyle'] = df['HouseStyle'].replace(style)

In [44]:
final_df['StyleScore_weight'] = np.multiply(df['N_HouseStyle'],df['ln_price'])

In [45]:
## going to readd some features
df_hope = final_df.select_dtypes(include=['float','int']).dropna(axis=1)

y = df_hope['ln_price']
X = df_hope.drop(columns=['SalePrice','ln_price','Id',
                         'BsmtFullBath','BsmtHalfBath','FullBath',
                         'HalfBath','1stFlrSF','2ndFlrSF',
                         ])

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=.8,test_size=.2,random_state=42)

model = LinearRegression()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test,y_pred)
rmse = sqrt(mse)
r2 = r2_score(y_test,y_pred)

beta_0 = model.intercept_
beta_i = model.coef_

print('mean squared error: ', mse)
print('root mean squared error: ', rmse)
print('R squared: ', r2)
print('---------------------')
print('Intercept: ', beta_0)
print('Coefficients: ', beta_i)

mean squared error:  0.009633099330455605
root mean squared error:  0.09814835368184024
R squared:  0.9483794131266572
---------------------
Intercept:  10.192370597058979
Coefficients:  [ 8.89194230e-05  5.99756958e-07  8.81760222e-02  5.89166873e-02
  4.49608698e-04  8.72654942e-04  3.08629144e-05  3.29227784e-05
  1.98203275e-05  8.36059605e-05  1.12304126e-05 -6.51878134e-06
 -2.03837812e-03 -6.76055818e-03  1.03978159e-02  2.41161130e-02
  3.14441032e-02  4.28992794e-06  6.45490431e-05 -4.17811410e-05
  1.13968844e-04  2.39767866e-04  9.07352582e-05 -2.03635604e-04
  9.73970065e-07  6.11008290e-04 -8.47267312e-04  2.23891363e-02
 -1.77492900e-05 -1.62755745e+01 -4.84000601e+00  4.04423632e-01]


# Stretch Goals

- Write a blog post explaining one of today's topics.
- Find a new regression dataset from the UCI machine learning repository and use it to test out your new modeling skillz.
 [ - UCI Machine Learning Repository - Regression Datasets](https://)
- Make a list for yourself of common feature engineering techniques. Browse Kaggle kernels to learn more methods.
- Start studying for tomorrow's topic: Gradient Descent
- Try and make the ultimate model with this dataset. clean as many features as possible, engineer the most sensible features as possible and see how accurate of a prediction you can make. 
- Learn about the "Dummy Variable Trap" and how it applies to linear regression modeling.
- Learning about using linear regression to model time series data